In [9]:
import os
import yaml
import numpy as np

!pwd

def print_result(algo, target, mode):
    if mode == 'continual': 
        file = 'global.yaml'
        index=-1
        print_ci95 = False
    else:
        file = f'global-{mode}.yaml'
        index='all'
        print_ci95 = True
    exp_path = f'/mnt/d/Documents/Github/CODA-Prompt-experiments/CGQA/10-task/{algo}/results-{target}/{file}'
    with open(exp_path, 'r') as yaml_file:
        yaml_result = yaml.safe_load(yaml_file)
    
    if index == 'all':
        data = np.asarray(yaml_result['mean'])      # [300]
    else:
        data = np.asarray(yaml_result['mean'][index])      # [-1]
    avg = np.mean(data)
    if print_ci95:
        std = np.std(data)
        ci95 = 1.96 * (std/np.sqrt(len(data)))
        print(f'{avg:.2f}% +- {ci95:.2f}%')
    else:
        ci95 = 0
        print(f'{avg:.2f}%')
    return avg, ci95

algo = 'coda-p-r-0'         # coda-p dual-prompt l2p++ coda-p-r
target = 'acc'
for mode in ['continual', 'sys', 'pro', 'sub', 'non', 'noc']:      # 
    print(f'{mode}:')
    print_result(algo, target, mode)



/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/mnt/d/Documents/Github/CODA-Prompt
continual:
55.36%
sys:
71.72% +- 0.76%
pro:
71.37% +- 0.69%
sub:
81.45% +- 0.61%
non:
74.55% +- 0.69%
noc:
84.07% +- 0.59%


In [1]:
%load_ext tensorboard

ModuleNotFoundError: No module named 'tensorboard'